In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

from matplotlib import rc
rc("font", family = "Malgun Gothic")

get_ipython().run_line_magic("matplotlib", "inline")

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
from tqdm.notebook import tqdm

from glob import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
Stop_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=1)
Route_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2019 = pd.read_excel("./2019.03/201903_2_도시철도.xlsx", sheet_name=4)

Stop_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=1)
Route_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2020 = pd.read_excel("./2020.03/202003_2_도시철도.xlsx", sheet_name=4)

Stop_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=0)
Xfer_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=1)
Route_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=2)
RouteStop_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=3)
StopTime_rail_2021 = pd.read_excel("./2021.03/202103_2_도시철도.xlsx", sheet_name=4)

# 1. 노선별 하루 운행 회수 구하기
- 1) Stop_rail과 StopTime 데이터를 "Stop_ID" 를 기준으로 병합
- 2) 필요없는 컬럼 제거
- 3) 피벗테이블을 사용해 노선별 운행 회수 계산

In [3]:
me_2021 = pd.merge(Stop_rail_2021, StopTime_rail_2021, on="Stop_ID", how="right")
me_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2021 = pd.DataFrame(pd.pivot_table(data=me_2021, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2021

,Stop_ID,Stop_NM,RT_ID,Arr_Time
0,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1D,125
1,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1U,130
2,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2D,43
3,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2U,51
4,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-5D,26
...,...,...,...,...
2390,RS_ACC1_S-5-0120,노은,RR_ACC1_S-5-01-2U,114
2391,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-1D,116
2392,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-2U,114
2393,RS_ACC1_S-5-0122,반석,RR_ACC1_S-5-01-1D,116


In [4]:
me_2020 = pd.merge(Stop_rail_2020, StopTime_rail_2020, on="Stop_ID", how="right")
me_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2020 = pd.DataFrame(pd.pivot_table(data=me_2020, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2020

,Stop_ID,Stop_NM,RT_ID,Arr_Time
0,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1D,128
1,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1U,133
2,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2D,98
3,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2U,99
4,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-5D,23
...,...,...,...,...
2225,RS_ACC1_S-5-0120,노은,RR_ACC1_S-5-01-2U,119
2226,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-1D,120
2227,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-2U,119
2228,RS_ACC1_S-5-0122,반석,RR_ACC1_S-5-01-1D,120


In [5]:
me_2019 = pd.merge(Stop_rail_2019, StopTime_rail_2019, on="Stop_ID", how="right")
me_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y', 'Gate'], inplace=True) 

rail_stop_count_2019 = pd.DataFrame(pd.pivot_table(data=me_2019, index=['Stop_ID', 'Stop_NM', "RT_ID"],values="Arr_Time",aggfunc=len)).reset_index()
rail_stop_count_2019

,Stop_ID,Stop_NM,RT_ID,Arr_Time
0,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1D,133
1,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-1U,141
2,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2D,124
3,RS_ACC1_S-1-0150,서울역(지하),RR_ACC1_S-1-01-2U,119
4,RS_ACC1_S-1-0151,시청,RR_ACC1_S-1-01-1D,133
...,...,...,...,...
2155,RS_ACC1_S-5-0120,노은,RR_ACC1_S-5-01-2U,119
2156,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-1D,120
2157,RS_ACC1_S-5-0121,지족,RR_ACC1_S-5-01-2U,119
2158,RS_ACC1_S-5-0122,반석,RR_ACC1_S-5-01-1D,120


# 2. 지역별 지하철역 구분하기
- 통행량 데이터의 지역구분에 따른다.
- 크롤링으로 구한 지하철역의 주소 데이터 사용


In [6]:
order_list = ['경기도 가평군', '경기도 고양시 ', '경기도 과천시', '경기도 광명시', '경기도 광주시', '경기도 구리시',
       '경기도 군포시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 부천시', '경기도 성남시 ',
       '경기도 수원시 ', '경기도 시흥시', '경기도 안산시 ', '경기도 안성시', '경기도 안양시 ', '경기도 양주시',
       '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시 ', '경기도 의왕시',
       '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 하남시',
       '경기도 화성시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구',
       '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구',
       '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구',
       '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구',
       '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구',
       '서울특별시 중랑구', '인천광역시 강화군', '인천광역시 계양구', '인천광역시 남동구', '인천광역시 동구',
       '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 옹진군',
       '인천광역시 중구']

In [7]:
rail_stop = pd.read_csv('./final_data/전철역주소통합본.csv', index_col=0)
rail_stop['지역'] = rail_stop['광역시/도'] + " " + rail_stop['시/군']
rail_stop.sort_values(by="지역")
rail_stop.head()

,Stop_NM,Lati,Long,광역시/도,시/군,구,동/읍/면,지역
0,서울역(지하),37.55587,126.97210,서울특별시,중구,중구,봉래동2가,서울특별시 중구
1,시청,37.56571,126.97712,서울특별시,중구,중구,정동,서울특별시 중구
2,종각,37.57016,126.98292,서울특별시,종로구,종로구,종로1가,서울특별시 종로구
3,종로3가,37.57042,126.99211,서울특별시,종로구,종로구,종로3가,서울특별시 종로구
4,종로5가,37.57093,127.00185,서울특별시,종로구,종로구,종로5가,서울특별시 종로구


# 3. 역별 호선, 칸 개수 
- 모든 정류장의 량수 필요: Stop파일, Route파일 사용해야 함 / 둘을 바로 병합할수 없어서 중간에 RouteStop 을 사용함
- 1) Stop_rail 파일과 RouteStop 파일을 "Stop_ID" 를 기준으로 병합 
- 2) 필요없는 컬럼 제거
- 3) 다시  Route_rail 파일과 "RT_ID" 를 기준으로 병합 (Route_rail 파일에 호선별 칸수 데이터가 있음)

In [8]:
line_2021 = pd.merge(Stop_rail_2021, RouteStop_rail_2021, on="Stop_ID", how="left")
line_2021.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2021 = pd.merge(line_2021, Route_rail_2021, on="RT_ID", how="left")
line_2021.fillna(0, inplace=True)
line_2021= line_2021[line_2021['객실수'] != 0]
line_2021

,Stop_ID,Stop_NM,Lati,Long,RT_ID,RT_Stop_Ord,Data_SEQ,Month,MTC_Region,RT_NM,RT_Desc,객실수
0,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1D,34,1,202103,1_수도권,서울1호선,서울1호선(소요산-인천)<하행>,10.0
1,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1U,29,2,202103,1_수도권,서울1호선,서울1호선(소요산-인천)<상행>,10.0
2,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2D,15,3,202103,1_수도권,서울1호선,서울1호선(광운대-신창)<하행>,10.0
3,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2U,43,4,202103,1_수도권,서울1호선,서울1호선(광운대-신창)<상행>,10.0
4,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-5D,10,9,202103,1_수도권,서울1호선,서울1호선(청량리-신창급행)<하행>,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3044,RS_ACC1_S-1-5004,파라다이스시티,37.437243,126.459759,RR_ACC1_S-1-ET-1U,3,63,202103,1_수도권,자기부상열차,자기부상열차<상행>,2.0
3045,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1D,5,62,202103,1_수도권,자기부상열차,자기부상열차<하행>,2.0
3046,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1U,2,63,202103,1_수도권,자기부상열차,자기부상열차<상행>,2.0
3047,RS_ACC1_S-1-5006,용유,37.424829,126.423629,RR_ACC1_S-1-ET-1D,6,62,202103,1_수도권,자기부상열차,자기부상열차<하행>,2.0


In [9]:
line_2020 = pd.merge(Stop_rail_2020, RouteStop_rail_2020, on="Stop_ID", how="left")
line_2020.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2020 = pd.merge(line_2020, Route_rail_2020, on="RT_ID", how="left")
line_2020.fillna(0, inplace=True)
line_2020= line_2020[line_2020['객실수'] != 0]
line_2020

,Stop_ID,Stop_NM,Lati,Long,RT_ID,RT_Stop_Ord,Data_SEQ,Month,MTC_Region,RT_NM,RT_Desc,객실수
0,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1D,34,1,202003,1_수도권,서울1호선,서울1호선(소요산-인천)<하행>,10.0
1,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1U,29,2,202003,1_수도권,서울1호선,서울1호선(소요산-인천)<상행>,10.0
2,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2D,15,3,202003,1_수도권,서울1호선,서울1호선(광운대-신창)<하행>,10.0
3,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2U,44,4,202003,1_수도권,서울1호선,서울1호선(광운대-신창)<상행>,10.0
4,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-5D,10,9,202003,1_수도권,서울1호선,서울1호선(청량리-신창급행)<하행>,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2834,RS_ACC1_S-1-5004,파라다이스시티,37.437243,126.459759,RR_ACC1_S-1-ET-1U,3,63,202003,1_수도권,자기부상열차,자기부상열차<상행>,2.0
2835,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1D,5,62,202003,1_수도권,자기부상열차,자기부상열차<하행>,2.0
2836,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1U,2,63,202003,1_수도권,자기부상열차,자기부상열차<상행>,2.0
2837,RS_ACC1_S-1-5006,용유,37.424829,126.423629,RR_ACC1_S-1-ET-1D,6,62,202003,1_수도권,자기부상열차,자기부상열차<하행>,2.0


In [10]:
line_2019 = pd.merge(Stop_rail_2019, RouteStop_rail_2019, on="Stop_ID", how="left")
line_2019.drop(columns=['Data_SEQ_x', 'Data_SEQ_y', 'Month_x', 'Month_y',"Gate"], inplace=True)
line_2019 = pd.merge(line_2019, Route_rail_2019, on="RT_ID", how="left")
line_2019.fillna(0, inplace=True)
line_2019= line_2019[line_2019['객실수'] != 0]
line_2019

,Stop_ID,Stop_NM,Lati,Long,RT_ID,RT_Stop_Ord,Data_SEQ,Month,MTC_Region,RT_NM,RT_Desc,객실수
0,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1D,34,1,201903,1_수도권,서울1호선,서울1호선(소요산-인천)<하행>,10.0
1,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-1U,29,2,201903,1_수도권,서울1호선,서울1호선(소요산-인천)<상행>,10.0
2,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2D,15,3,201903,1_수도권,서울1호선,서울1호선(광운대-신창)<하행>,10.0
3,RS_ACC1_S-1-0150,서울역(지하),37.555870,126.972100,RR_ACC1_S-1-01-2U,44,4,201903,1_수도권,서울1호선,서울1호선(광운대-신창)<상행>,10.0
4,RS_ACC1_S-1-0151,시청,37.565710,126.977120,RR_ACC1_S-1-01-1D,33,1,201903,1_수도권,서울1호선,서울1호선(소요산-인천)<하행>,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1696,RS_ACC1_S-1-5004,파라다이스시티,37.437243,126.459759,RR_ACC1_S-1-ET-1U,3,61,201903,1_수도권,자기부상열차,자기부상열차<상행>,2.0
1697,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1D,5,60,201903,1_수도권,자기부상열차,자기부상열차<하행>,2.0
1698,RS_ACC1_S-1-5005,워터파크,37.429122,126.434066,RR_ACC1_S-1-ET-1U,2,61,201903,1_수도권,자기부상열차,자기부상열차<상행>,2.0
1699,RS_ACC1_S-1-5006,용유,37.424829,126.423629,RR_ACC1_S-1-ET-1D,6,60,201903,1_수도권,자기부상열차,자기부상열차<하행>,2.0


# 4. 데이터 합치기(연도별로)
- 위에서 만든 모든 파일을 지하철 역명을 기준으으로 합쳐준다.

In [11]:
tmp21 = pd.merge(rail_stop_count_2021, line_2021,on=['RT_ID', "Stop_NM"], how='outer')
tmp21.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
pd.merge(tmp21, rail_stop, on='Stop_NM', how='left').to_excel('./aa/21년전철.xlsx', encoding='utf-8')

tmp20 = pd.merge(rail_stop_count_2020, line_2020,on=['RT_ID', "Stop_NM"], how='outer')
tmp20.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
pd.merge(tmp20, rail_stop, on='Stop_NM', how='left').to_excel('./aa/20년전철.xlsx', encoding='utf-8')

tmp19 = pd.merge(rail_stop_count_2019, line_2019,on=['RT_ID', "Stop_NM"], how='outer')
tmp19.drop(columns=['Stop_ID_x', "Data_SEQ", "Month", "RT_Stop_Ord"], inplace=True)
pd.merge(tmp20, rail_stop, on='Stop_NM', how='left').to_excel('./aa/19년전철.xlsx', encoding='utf-8')